# Packages

In [1]:
import pandas as pd
import glob
import ast
import os
from tqdm import tqdm
import numpy as np
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

# Functions

In [2]:
def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

In [3]:
def remove_outliers(df):
    
    feature_titles = ['Room Count', 'Floor Count', 'Price', 'Interior Area']

    outlier_indexes = []
    for title in tqdm(feature_titles):
        q25, q75 = np.nanpercentile(df[title], 25), np.nanpercentile(df[title], 75)
        iqr = q75 - q25

        cut_off = iqr * 1.5
        lower, upper = q25 - cut_off, q75 + cut_off

        outliers_indices = [count for count, x in enumerate(df[title]) if x < lower or x > upper]
        outlier_indexes.append(outliers_indices)
        
        print('Identified outliers: %d' % len(outliers_indices))

        if len(outliers_indices) == 0:
            print('Non-outlier observations: %d' % df.shape[0])
            print('________________________________')
            outliers_indices = []
            continue
        
        else:
            print('Non-outlier observations: %d' % (df.shape[0] - len(outliers_indices)))
            print('________________________________')
            outliers_indices = []

    outlier_indexes = list(set(flatten(outlier_indexes)))    
    indexes_to_keep = set(range(df.shape[0])) - set(outlier_indexes)
    df_no_outliers = df.take(list(indexes_to_keep))
    
    return df_no_outliers

# Preprocessing

## With Facilities (w/o list.am)

### Concatenate

In [4]:
csvs = ['bnakaran.csv', 'DataRealEstate.csv', 'EstateAmData.csv', 'MyRealty.csv']

path = os.getcwd()
all_files = glob.glob(os.path.join(path, "*.csv"))

desired_files = []
for csv in all_files:
    for name in csvs:
        if name in csv:
            desired_files.append(csv)
        else:
            continue

li = []
for filename in desired_files:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.drop(columns=['Additional Info'], inplace=True)

for count, row in enumerate(frame['Facilities']):
    if row != row:
        continue
    else:
        frame['Facilities'][count] = ast.literal_eval(row)

for count, row in enumerate(frame['Building Material']):
    if row != row:
        continue
    else:
        frame['Building Material'][count] = row.lower()

### Mapping

In [5]:
all_facilities = []
for row in frame['Facilities']:
    if row != row:
        continue 
    for i in range(len(row)):
        all_facilities.append(row[i])

In [6]:
all_materials = []
for row in frame['Building Material']:
    if row != row:
        continue 
    else:
        all_materials.append(row)

In [7]:
Counter(all_facilities).most_common

<bound method Counter.most_common of Counter({'Electricity': 12319, 'Water': 12236, 'Gas': 11139, 'Heating': 10220, 'Hot water': 10058, 'Internet': 8468, 'Air-conditioner': 7897, 'water 24/7': 5658, 'Standing water': 5190, 'Sewerage, Canalization': 2386, 'Natural gas': 1114, 'Central heating': 948, 'Irrigation': 360, 'Persistent water': 210, 'Open balcony': 130, 'Euro windows': 119, 'Sunny': 114, 'Elevator': 104, 'Iron door': 99, 'View': 97, 'Close to the bus station': 69, 'TV': 56, 'Roadside': 51, 'Tile': 41, 'Park': 40, 'Cable TV': 40, 'Bilateral': 38, 'Playground': 35, 'Furniture': 35, 'Internet, Wi-Fi': 31, 'Equipment': 28, 'TV, cable': 27, 'Balcony': 23, 'Laminate flooring': 20, 'Parquet': 20, 'Parking': 19, 'Garage': 15, 'Security system': 15, 'Basement': 14, 'Loggia': 8, 'Storage room': 7, 'High first floor': 7, 'Fence': 5, 'Grating': 5, 'Swimming pool': 5, 'TV, cable, satellite': 4, 'TV, satellite': 4, 'Heated floor': 4, 'Gate': 3, 'Attic': 3, 'Gym': 3, 'Fireplace': 2, 'Sauna':

In [8]:
Counter(all_materials).most_common

<bound method Counter.most_common of Counter({'stone': 6424, 'monolit': 4166, 'panel': 2596, 'other': 813, 'concrete': 599, 'wall building': 169, 'մոնոլիտ': 1, 'քարե': 1, 'монолит': 1})>

In [9]:
d = dict()

d['Natural gas'] = ('Gas Available')
d['Gas'] = ('Gas Available')

d['Internet'] = ('Internet Available')
d['Internet, Wi-Fi'] = ('Internet Available')

d['water 24/7'] = ('Constant Water')
d['Persistent water'] = ('Constant Water')
d['Standing water'] = ('Constant Water')

d['Sewerage, Canalization'] = ('Sewerage')

d['stone'] = 'Stone'
d['քարե'] = 'Stone'

d['monolit'] = 'Monolit'
d['concrete'] = 'Monolit'
d['մոնոլիտ'] = 'Monolit'
d['монолит'] = 'Monolit'

d['wall building'] = 'Panel'
d['panel'] = 'Panel'

In [10]:
for count, row in enumerate(frame['Building Material']):
    if row in d:
        frame['Building Material'][count] = d[row]
    else:
        continue

In [11]:
for count, row in enumerate(frame['Facilities']):
    if row != row:
        continue
    for i in range(len(row)):
        if row[i] in d:
            frame['Facilities'][count][i] = d[row[i]]
        else:
            continue

### Add Columns & One Hot Encode

In [12]:
frame = frame.reindex(columns = frame.columns.tolist() + ['Electricity', 'Water', 'Gas Available',
                                                         'Heating', 'Hot water', 'Internet Available',
                                                         'Air-conditioner', 'Constant Water', 'Sewerage'])


In [13]:
for counter, row in enumerate(frame['Facilities']):
    if row != row:
        continue  
    for i in range(len(row)):
        if 'Electricity' in row[i]:
            frame['Electricity'][counter] = 1
        elif 'Water' in row[i]:
            frame['Water'][counter] = 1
        elif 'Gas Available' in row[i]:
            frame['Gas Available'][counter] = 1
        elif 'Heating' in row[i]:
            frame['Heating'][counter] = 1
        elif 'Hot water' in row[i]:
            frame['Hot water'][counter] = 1
        elif 'Internet Available' in row[i]:
            frame['Internet Available'][counter] = 1
        elif 'Air-conditioner' in row[i]:
            frame['Air-conditioner'][counter] = 1
        elif 'Constant Water' in row[i]:
            frame['Constant Water'][counter] = 1
        elif 'Sewerage' in row[i]:
            frame['Sewerage'][counter] = 1
        else:
            if i == len(row):
                frame['Electricity'][counter] = 0
                frame['Water'][counter] = 0
                frame['Gas Available'][counter] = 0
                frame['Heating'][counter] = 0
                frame['Hot water'][counter] = 0
                frame['Internet Available'][counter] = 0
                frame['Air-conditioner'][counter] = 0
                frame['Constant Water'][counter] = 0
                frame['Sewerage'][counter] = 0
            else:
                continue

In [14]:
frame = frame.join(pd.get_dummies(frame['Building Material'], prefix='Material'))

### Outlier Detection

In [15]:
houses = frame[frame['Product Type'] == 'house']
apartments = frame[frame['Product Type'] == 'apartment']
houses.reset_index(drop=True, inplace=True)
apartments.reset_index(drop=True, inplace=True)

In [16]:
print(apartments.sort_values(by=['Price'], ascending=False)['Price'].head(5))
print(houses.sort_values(by=['Price'], ascending=False)['Price'].head(5))

2255    106000000
7123     65625000
7039     39000000
7086     36750000
7038     32550000
Name: Price, dtype: int64
1627    5000000
205     3000000
155     2750000
22      2700000
2999    2500000
Name: Price, dtype: int64


In [17]:
apartments_no_outliers = remove_outliers(apartments)

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 285.64it/s]

Identified outliers: 484
Non-outlier observations: 10679
________________________________
Identified outliers: 0
Non-outlier observations: 11163
________________________________
Identified outliers: 940
Non-outlier observations: 10223
________________________________
Identified outliers: 675
Non-outlier observations: 10488
________________________________


In [18]:
houses_no_outliers = remove_outliers(houses)

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 800.25it/s]

Identified outliers: 37
Non-outlier observations: 3570
________________________________
Identified outliers: 30
Non-outlier observations: 3577
________________________________
Identified outliers: 293
Non-outlier observations: 3314
________________________________
Identified outliers: 193
Non-outlier observations: 3414
________________________________


In [19]:
print(apartments_no_outliers.sort_values(by=['Price'], ascending=False)['Price'].head(5))
print(houses_no_outliers.sort_values(by=['Price'], ascending=False)['Price'].head(5))

8648    315000
9740    315000
3075    315000
3786    315000
7894    315000
Name: Price, dtype: int64
2213    790000
583     790000
195     770000
774     770000
1       770000
Name: Price, dtype: int64


### Concatenate

In [20]:
full_no_outliers = apartments_no_outliers.append(houses_no_outliers, ignore_index = True)

### Drop Redundancies & Store

In [21]:
full_no_outliers.drop(columns=['Facilities', 'Building Material'], inplace=True)

In [22]:
full_no_outliers.to_csv('dataFacilities.csv')

## Without Facilities (full data)

### Concatenate

In [23]:
csvs = ['bnakaran.csv', 'DataRealEstate.csv', 'EstateAmData.csv', 'MyRealty.csv', 'list.csv']

path = os.getcwd()
all_files = glob.glob(os.path.join(path, "*.csv"))

desired_files = []
for csv in all_files:
    for name in csvs:
        if name in csv:
            desired_files.append(csv)
        else:
            continue

li = []
for filename in desired_files:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.drop(columns=['Additional Info', 'Facilities'], inplace=True)

for count, row in enumerate(frame['Building Material']):
    if row != row:
        continue
    else:
        frame['Building Material'][count] = row.lower()

### Mapping

In [24]:
all_materials = []
for row in frame['Building Material']:
    if row != row:
        continue 
    else:
        all_materials.append(row)

In [25]:
Counter(all_materials).most_common

<bound method Counter.most_common of Counter({'stone': 27130, 'panels': 8594, 'monolith': 5577, 'monolit': 4166, 'panel': 2596, 'other': 813, 'concrete': 599, 'wall building': 169, 'cassette': 141, 'bricks': 107, 'wooden': 71, 'մոնոլիտ': 1, 'քարե': 1, 'монолит': 1})>

In [26]:
d = dict()

d['stone'] = 'Stone'
d['քարե'] = 'Stone'

d['monolit'] = 'Monolit'
d['concrete'] = 'Monolit'
d['մոնոլիտ'] = 'Monolit'
d['монолит'] = 'Monolit'
d['monolith'] = 'Monolit'

d['wall building'] = 'Panel'
d['panel'] = 'Panel'
d['panels'] = 'Panel'

In [27]:
for count, row in enumerate(frame['Building Material']):
    if row in d:
        frame['Building Material'][count] = d[row]
    else:
        continue

### One Hot Encode

In [28]:
frame = frame.join(pd.get_dummies(frame['Building Material'], prefix='Material'))

### Outlier Detection

In [29]:
houses = frame[frame['Product Type'] == 'house']
apartments = frame[frame['Product Type'] == 'apartment']
houses.reset_index(drop=True, inplace=True)
apartments.reset_index(drop=True, inplace=True)

In [30]:
print(apartments.sort_values(by=['Price'], ascending=False)['Price'].head(5))
print(houses.sort_values(by=['Price'], ascending=False)['Price'].head(5))

20342    145000000.0
2255     106000000.0
23491     69600000.0
9856      69500000.0
7123      65625000.0
Name: Price, dtype: float64
4547     2.500000e+09
16475    3.500000e+08
15892    3.250000e+08
15368    1.800000e+08
9406     1.800000e+08
Name: Price, dtype: float64


In [31]:
houses_no_outliers = remove_outliers(houses)

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 231.57it/s]

Identified outliers: 37
Non-outlier observations: 18289
________________________________
Identified outliers: 742
Non-outlier observations: 17584
________________________________
Identified outliers: 1587
Non-outlier observations: 16739
________________________________
Identified outliers: 1380
Non-outlier observations: 16946
________________________________


In [32]:
apartments_no_outliers = remove_outliers(apartments)

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 151.91it/s]

Identified outliers: 1004
Non-outlier observations: 30636
________________________________
Identified outliers: 0
Non-outlier observations: 31640
________________________________
Identified outliers: 2595
Non-outlier observations: 29045
________________________________
Identified outliers: 1648
Non-outlier observations: 29992
________________________________


In [33]:
print(apartments_no_outliers.sort_values(by=['Price'], ascending=False)['Price'].head(5))
print(houses_no_outliers.sort_values(by=['Price'], ascending=False)['Price'].head(5))

3591     250000.0
4722     250000.0
30247    250000.0
1273     250000.0
3939     250000.0
Name: Price, dtype: float64
4679     540500.0
13723    540000.0
3626     540000.0
15418    540000.0
1133     540000.0
Name: Price, dtype: float64


### Concatenate

In [34]:
full_no_outliers = apartments_no_outliers.append(houses_no_outliers, ignore_index = True)

### Drop Redundancies & Store

In [35]:
full_no_outliers.drop(columns=['Building Material'], inplace=True)

In [36]:
full_no_outliers.to_csv('dataFullNoFacilities.csv')